In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
path= get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text= io.open(path, encoding='utf-8').read().lower()
print ('corpus length: ', len(text))

corpus length:  600893


In [3]:
chars= sorted(list(set(text)))
print ('total chars: ', len(chars))

total chars:  57


In [4]:
char_indices= dict((c,i) for i,c in enumerate(chars))
indices_char= dict((i,c) for i,c in enumerate(chars))

In [5]:
max_len=40
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-max_len, step):
    sentences.append(text[i:i+max_len])
    next_chars.append(text[i+max_len])
print ('nb sequences: ', len(sentences))

nb sequences:  200285


In [6]:
print ('Vectorization....')
x=np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
print (x.shape,y.shape)

Vectorization....
(200285, 40, 57) (200285, 57)


In [7]:
for i,sentence in enumerate(sentences):
    for t,char in  enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]] = 1


In [8]:
model=Sequential()
model.add(LSTM(128, input_shape=(max_len,len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer= RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [ ]:
def sample(preds,temperature=1):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    print (preds)
    exp_preds=np.exp(preds)
    print exp_preds
    probas=np.random.multinomial(1,preds,1)
    return 
    return np.argmax(probas)

In [10]:
model.fit(x,y, batch_size=128, epochs=1)
start_index= random.randint(0,len(text)-max_len-1)
sentence=text[start_index:start_index+max_len]
print ('Generated with seed: "'+sentence + '"')
x_pred= np.zeros((1,max_len,len(chars)))
for t, char in enumerate(sentence):
        x_pred[0,t,char_indices[char]]=1.
preds=model.predict(x_pred, verbose=0)[0]
print (preds.shape)
print (preds)

Epoch 1/1
   640/200285 [..............................] - ETA: 27:38:00 - loss: 3.7554

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.453544). Check your callbacks.
  % delta_t_median)


200285/200285 [==============================] - 417s 2ms/step - loss: 1.9884
Generated with seed: "an eighty thousand years old,
we should "
(57,)
[  1.05222245e-03   1.64155941e-03   2.00016162e-04   5.44730527e-03
   1.31998197e-04   1.62742144e-04   2.28950463e-04   1.16799714e-03
   6.98311778e-04   7.66837271e-04   4.28954954e-05   1.92217587e-04
   1.52955385e-04   7.34341957e-05   1.39585914e-04   9.01305320e-05
   3.91679096e-05   1.82263775e-05   5.03812298e-05   6.96826974e-05
   4.87433921e-04   3.64427629e-04   2.90805940e-04   2.72143545e-04
   7.89426849e-05   5.46885203e-05   9.46414657e-05   1.17685504e-01
   6.18634596e-02   2.75281928e-02   4.70154956e-02   3.25275250e-02
   4.41162921e-02   1.77386869e-02   3.56434286e-02   7.90821835e-02
   1.88642740e-03   7.43590482e-03   2.07994934e-02   4.51101549e-02
   4.58010733e-02   7.60133490e-02   4.92234677e-02   1.11962471e-03
   3.43914106e-02   4.56090719e-02   1.33934587e-01   1.16051110e-02
   9.08215903e-03   3.01

In [ ]:
for i in range(1,50):
    print('-'*50)
    print ('iteration: ',i)
    model.fit(x,y, batch_size=128, epochs=1)
    start_index= random.randint(0,len(text)-max_len-1)
    
    for diversity in [0.2,0.5,1.0,1.2]:
        print('Diversity: ',diversity)
        generated=''
        sentence=text[start_index:start_index+max_len]
        generated+=sentence
        print ('Generated with seed: "'+sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x_pred= np.zeros((1,max_len,len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0,t,char_indices[char]]=1.
            
            preds=model.predict(x_pred, verbose=0)[0]
            next_index= sample(preds, diversity)
            next_char=indices_char[next_index]
            
            generated+=next_char
            sentence=sentence[1:]+next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()